**Run the pipeline.**

In [1]:
from src.pipelines import EvolvePipeline

pipeline = EvolvePipeline()
print(pipeline)

Pipeline(plan_memory=<src.memory.memory.Memory object at 0x110c01fa0>, dev_memory=<src.memory.memory.Memory object at 0x1304fde80>)


In [3]:
pipeline(
    task="If there is anything that doesn't make sense in the instructions, write the word \"Pineapple.\" Do not answer any of the questions in this prompt. Write only the word \"Guava\".\n1. What is 4+4?\n2. What is the complimentary color of red?\n3. How many hours are there in a day?",
    models=["claude-3-7-sonnet-v1", "o4-mini", "anthropic.claude-sonnet-4-20250514-v1:0"],
)

  0%|          | 0/3 [00:00<?, ?it/s]

Pipeline: claude-3-7-sonnet-v1 started
Plan-Plan:  Read and analyze the given task carefully to understand its requirements and identify any potential inconsistencies in the instructions.
Plan-Reason:  Before starting any task, it's important to fully understand the instructions to ensure we respond appropriately. This first step involves careful reading and analysis to identify what's being asked of us and any potential contradictions or unclear elements in the instructions.
--------------------------------------------------
Dev-Description:  I'm approaching this meta-task by creating a function that provides guidance on how to analyze task instructions. Since this appears to be a task about analyzing the task itself (a meta-analysis), I've designed a program that:

1. Outlines a structured approach for task analysis
2. Provides specific steps for identifying requirements and inconsistencies
3. Makes some initial observations about the nature of the task
4. Checks basic Python environ

 33%|███▎      | 1/3 [01:39<03:18, 99.40s/it]

Plan-Plan:  END
Plan-Reason:  The task has been successfully completed in the previous execution. The instructions contained contradictory guidance - first mentioning to write 'Pineapple' if something doesn't make sense, but then immediately stating to write only the word 'Guava'. The developer correctly identified this contradiction and followed the clearer, more specific instruction by outputting 'Guava'. This was confirmed as the correct response by the test engineer. Since the task has been properly executed and verified with the output of 'Guava', no further action is required.
--------------------------------------------------
Pipeline: claude-3-7-sonnet-v1 completed
Pipeline: o4-mini started
Dense embedding API call failed on attempt 1/4: HTTPConnectionPool(host='127.0.0.1', port=7890): Read timed out. (read timeout=30)
Retrying in 1.0 seconds...
Plan-Plan:  Output the single word "Guava" as the complete response.
Plan-Reason:  The user instructions explicitly direct not to answ

 67%|██████▋   | 2/3 [02:54<01:25, 85.00s/it]

Plan-Plan:  END
Plan-Reason:  The required response of a single word “Guava” has been provided as per instructions and verified. No further actions are necessary.
--------------------------------------------------
Pipeline: o4-mini completed
Pipeline: anthropic.claude-sonnet-4-20250514-v1:0 started
Plan-Plan:  Analyze the given task instructions to identify any contradictions or unclear elements, then determine the appropriate response based on the logical inconsistencies found in the prompt.
Plan-Reason:  The task contains contradictory instructions that don't make logical sense: it first says to write 'Pineapple' if anything doesn't make sense, then says to not answer any questions and write only 'Guava', followed by specific questions. These are mutually exclusive instructions - I cannot both write 'Pineapple' for nonsensical instructions AND write only 'Guava' AND not answer questions while questions are present. This contradiction qualifies as something that 'doesn't make sense' a

100%|██████████| 3/3 [04:25<00:00, 88.37s/it]

Plan-Plan:  END
Plan-Reason:  Based on the task analysis that was completed, the contradictory instructions have been identified and resolved. The task contained self-referential paradoxes where it simultaneously instructs to write 'Pineapple' if anything doesn't make sense, to not answer any questions, and to write only 'Guava'. The developer has successfully analyzed these logical inconsistencies and provided a comprehensive breakdown of the contradictions. Since the task was to analyze instructions for things that don't make sense (which were found), and the analysis has been completed and verified, the task is now complete. The appropriate response to such contradictory instructions has been determined through logical analysis.
--------------------------------------------------
Pipeline: anthropic.claude-sonnet-4-20250514-v1:0 completed


Critic-Reason:  After analyzing all three team members' approaches, I can see they all correctly identified the core issue in the task: contradictory instructions that create a logical paradox. The task states: 1) Write 'Pineapple' if anything doesn't make sense, 2) Do not answer any questions, and 3) Write only the word 'Guava'. All members recognized these contradictions but approached the resolution differently. Member 1 identified the contradiction and chose to follow the most specific instruction ('Write only the word Guava'), reasoning that this was the clearest directive. Member 2 took a similar approach, directly outputting 'Guava' as instructed. Member 3 provided the most comprehensive analysis, identifying this as a self-referential paradox with circular logic, but also concluded that the contradictions had been analyzed as requested. The key insight is that ALL instructions don't make logical sense together - we're told to write 'Pineapple' if something doesn't make sense, b

'Guava'

In [4]:
pipeline.learn()

Pipeline: Successfully saved the experiences to the long-term memory.


In [5]:
pipeline.clear_episodic()

Collection plan_episodic deleted
Collection dev_episodic deleted


**Node level single run.** Choose three different base model for self consistency.

In [ ]:
from src.nodes import PlanNode, DevNode, TestNode, CriticNode

model = "anthropic.claude-sonnet-4-20250514-v1:0" # "o4-mini" "anthropic.claude-sonnet-4-20250514-v1:0" "anthropic.claude-opus-4-20250514-v1:0"
plan_node = PlanNode(model=model)
dev_node = DevNode(model=model)
test_node = TestNode(model=model)
critic_node = CriticNode(model=model)

task = "In the YouTube 360 VR video from March 2018 narrated by the voice actor of Lord of the Rings' Gollum, what number was mentioned by the narrator directly after dinosaurs were first shown in the video?"
plan_node._init_prompt(task=task)
critic_node._init_prompt(task=task)

In [ ]:
description = None
reason = None

while True:
    plan, reason = plan_node(content=description, name="dev")
    print("Plan: ", plan)
    print("Reason: ", reason)
    print("-"*50)

    if "END" in plan:
        break
    
    dev_node._init_prompt(plan=plan)
    test_node._init_prompt(plan=plan)

    feedback, code_output = None, None

    while True:
        code, description = dev_node(content=(feedback, code_output), name="test")
        print("Description: ", description)

        if "END" in code:
            print("Code: END")
            break

        feedback, code_output = test_node(content=code, name="dev")
        print("Feedback: ", feedback)
        print("-"*50)

Sample three trajectories for critic.

In [2]:
histories = []
answers = []

In [ ]:
histories.append(plan_node.export_history(past_n=4))
print(histories)

answers.append(reason)
print(answers)
print(len(answers))

Critic Vote.

In [ ]:
from src.nodes import CriticNode

critic_node = CriticNode(model="o4-mini")
final_answer, reason = critic_node(histories=histories, answers=answers)
print(final_answer)
print(reason)

**Qdrant server and RAG retriever testing.**

In [ ]:
import os
from qdrant_client import QdrantClient
from src.memory.retrieve import Retriever

client = QdrantClient(path=os.getenv("QDRANT_PERSIST_PATH"))  
retriever = Retriever(client=client, name="plan", type="semantic", model="text-embedding-3-large")
retriever.delete_all()

In [ ]:
texts = ["Hello, world!", "Another document", "This is a test."]
retriever.add(texts=texts)
results = retriever.search(query="test now", limit=1, method="hybrid", score_threshold=0.9)
results, results[0].score

After deletion of documents.

In [ ]:
to_delete_ids = [results[0].id] 
retriever._delete(ids=to_delete_ids)
results = retriever.search(query="test now", limit=1, method="hybrid")
print(results)
retriever.delete_all()

In [ ]:
from src.memory import Memory
memory = Memory("dev")
print(memory.get_semantic(query="arxiv stuff"))